<a href="https://colab.research.google.com/github/muharremosmantopakkaya/LSTM-STOCK-PRICE-PREDICTION/blob/master/LSTM-STOCK-PRICE-PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### LSTM KULLANARAK HİSSE SENEDİ FİYAT TAHMİNİ

Bu uygulama, finansal veri analizi için Python'da LSTM (Uzun Kısa Vadeli Bellek) modelinin kullanımını göstermektedir. Veri indirme, model oluşturma ve tahminlerin görselleştirilmesi adımlarını içermektedir. LSTM modeli, bir hisse senedinin kapanış fiyatlarını tahmin etmek için kullanılmıştır. Kod blokları, hisse senedi verilerinin alınması, modelin eğitilmesi ve tahminlerin çizimlerle görselleştirilmesini içermektedir.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import yfinance as yf

# Hisse senedi verisini alma

In [ ]:
ticker = 'THYAO.IS'
start_date = '2015-01-01'
end_date = '2023-01-01'
stock_data = yf.download(ticker, start=start_date, end=end_date)


# Kapanış fiyatlarını görselleştirme

In [ ]:
plt.style.use('dark_background')
plt.figure(figsize=(16, 8))
plt.title('Kapanış Fiyatı')
plt.plot(stock_data['Close'])
plt.xlabel('Tarih', fontsize=18)
plt.ylabel('Kapanış Fiyatı (TL)', fontsize=18)
plt.show()

# Veri hazırlama

In [ ]:
data = stock_data.filter(['Close'])
dataset = data.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)


# Eğitim veri uzunluğu

In [ ]:
training_data_len = int(len(dataset) * 0.8)
train_data = scaled_data[:training_data_len, :]
test_data = scaled_data[training_data_len - 60:, :]


# Eğitim veri seti oluşturma


In [ ]:
x_train, y_train = [], []
for i in range(60, len(train_data)):
    x_train.append(train_data[i - 60:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


# LSTM modeli oluşturma ve eğitme


In [ ]:
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=32, epochs=17)

# Modelin test edilmesi


In [ ]:
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i - 60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# Grafik çizimi

In [ ]:
train = data[:training_data_len]
valid = data[training_data_len:]
valid['predictions'] = predictions

plt.figure(figsize=(16, 8))
plt.title('Model Tahmin Sonuçları - Amazon Hisse Senedi')
plt.xlabel('Tarih', fontsize=18)
plt.ylabel('Kapanış Fiyatı (USD)', fontsize=18)

plt.plot(train['Close'], color='red', label='Eğitim Veri Seti')
plt.plot(valid['Close'], color='yellow', label='Test Veri Seti')
plt.plot(valid['predictions'], color='green', label='Tahminler')

plt.legend(loc='upper left')  # Legend konumunu değiştirilebilir

plt.show()

# Grafik çizimi

In [ ]:

train = data[:training_data_len]
valid = data[training_data_len:].copy()
valid['predictions'] = predictions

# Son 15 günü görüntüleme

In [ ]:
print(valid.tail(15))

# İleriye dönük 1 ay için fiyat tahmini yapma

In [ ]:
last_60_days = scaled_data[-60:]
X_test = np.array([last_60_days])
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

pred_price_1months = model.predict(X_test)
pred_price_1months = scaler.inverse_transform(pred_price_1months)
print(f'{ticker} için 1 ay sonraki fiyat: {pred_price_1months[0][0]}')